In [1]:
%matplotlib inline

## Remove black box around the lesion

In [2]:
image_rgb = io.imread("./data/blue-veil/ISIC_0054527.jpg")
io.imshow(image_rgb)

NameError: name 'io' is not defined

In [ ]:
from skimage.transform import rotate

def prepare_single_border_mask(binary):
    mask = np.zeros_like(binary)
    for i, row in enumerate(binary):
        fill_ones = False
        for j, cell in enumerate(row):
            if fill_ones:
                mask[i, j] = 1.0
                continue
            if 0.0 == cell:
                continue
            mask[i, j] = 1.0
            fill_ones = True
    return mask.astype(bool)

def prepare_border_mask(binary, dim=1):
    rotated_90 = rotate(binary, 90, preserve_range=True)
    rotated_180 = rotate(binary, 180, preserve_range=True)
    rotated_270 = rotate(binary, 270, preserve_range=True)
    # Create masks in each direction
    mask_0 = prepare_single_border_mask(binary)
    mask_90 = rotate(prepare_single_border_mask(rotated_90), -90, 
                     preserve_range=True)
    mask_180 = rotate(prepare_single_border_mask(rotated_180), -180,
                     preserve_range=True)
    mask_270 = rotate(prepare_single_border_mask(rotated_270), -270,
                      preserve_range=True)
    mask = (np.logical_and(mask_0, mask_90) & \
            np.logical_and(mask_180, mask_270)).astype(bool)
    return mask

In [ ]:
def get_average_colour(image_rgb, border_mask):
    masked_image_rgb = image_rgb[border_mask]
    r, g, b = np.mean(masked_image_rgb[:, 0]), \
        np.mean(masked_image_rgb[:, 1]), \
        np.mean(masked_image_rgb[:, 2])
    return np.array([r, g, b], dtype=np.uint8)

In [ ]:
from skimage.filters import threshold_otsu
from skimage.color import rgb2gray

def get_border_mask(image_rgb):
    image_gray = rgb2gray(image_rgb)
    thresh = threshold_otsu(image_gray)
    binary = (image_gray > thresh).astype(int)
    return prepare_border_mask(binary)

def remove_border(image_rgb):
    border_mask = get_border_mask(image_rgb)
    avg_colour = get_average_colour(image_rgb, border_mask)
    borderless_image_rgb = np.copy(image_rgb)
    borderless_image_rgb[~border_mask] = avg_colour
    return borderless_image_rgb

In [ ]:
borderless_image_rgb = remove_border(image_rgb)
io.imshow(borderless_image_rgb)

In [ ]:
def get_3d_border_mask(image_rgb):
    mask_2d = get_border_mask(image_rgb * 3)
    return np.dstack([mask_2d] * 3)

# Blue veil histogram encoding

In [ ]:
from skimage import io
from glob import glob

for img_path in glob("./data/blue-veil/*"):
    image_rgb = io.imread(img_path)
    io.imshow(image_rgb)
    break

In [ ]:
image_rgb.shape

In [ ]:
image_r = image_rgb[:, :, 0]
image_g = image_rgb[:, :, 1]
image_b = image_rgb[:, :, 2]

In [ ]:
from skimage.exposure import histogram

from matplotlib import pyplot as plt

def show_histogram(image_channel, **kwargs):
    hist, bin_centers = histogram(image_channel, nbins=256)
    plt.plot(bin_centers, hist, **kwargs)

In [ ]:
show_histogram(image_r, c="r", label="red")
show_histogram(image_g, c="g", label="green")
show_histogram(image_b, c="b", label="blue")
plt.legend()

In [ ]:
from skimage.color import rgb2hsv

image_hsv = rgb2hsv(image_rgb)
show_histogram(image_hsv[:, :, 0], label="hue")
show_histogram(image_hsv[:, :, 1], label="saturation")
show_histogram(image_hsv[:, :, 2], label="value")
plt.legend()

In [ ]:
import numpy as np

def vectorized_histogram(image_rgb):
    histograms = []
    for ch in range(image_rgb.shape[2]):
        mask_2d = get_border_mask(image_rgb)
        image_channel = image_rgb[:, :, ch][mask_2d]
        hist, _ = np.histogram(image_channel, density=True,
                               bins=256, range=(0,255))
        histograms.append(hist)
    return np.stack(histograms).reshape((-1, ))

In [ ]:
vectorized_histogram(image_rgb)

In [ ]:
import pandas as pd

filenames, dataset = [], []
for img_path in glob("./data/blue-veil/*"):
    try:
        image_rgb = io.imread(img_path)
        hist = vectorized_histogram(image_rgb)
        filenames.append(img_path)
        dataset.append(hist)
    except ValueError as e:
        print("Error while processing", img_path)

dataset_df = pd.DataFrame(dataset, index=filenames)
dataset_df.to_csv("./data/blue-veil.csv")
dataset_df.head()

In [ ]:
vector_length = dataset_df.iloc[0].shape[0] # should be 768

In [ ]:
from os import path

full_filenames, full_dataset = [], []
for img_path in glob("./data/ISIC_2019_Test_Input/*"):
    try:
        image_rgb = io.imread(img_path)
        hist = vectorized_histogram(image_rgb)
        full_filenames.append(img_path)
        full_dataset.append(hist)
    except ValueError as e:
        print("Error while processing", img_path)

full_dataset_df = pd.DataFrame(full_dataset, index=full_filenames)
full_dataset_df.to_csv("./data/ISIC_2019_Test_Input.csv")
full_dataset_df.head()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_validate = train_test_split(dataset_df, test_size=0.1, 
                                       random_state=2019)
X_train, X_test = train_test_split(X_train, test_size=0.25, 
                                   random_state=2020)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.models import Model

autoencoder = Sequential([
    Dense(64, input_shape=(vector_length, )),
    Activation("tanh"),
    Dense(64),
    Activation("tanh"),
    Dense(32),
    Activation("tanh"),
    Dense(16),
    Activation("tanh"),
    Dense(8), # coding layer
    Activation("tanh"),
    Dense(16),
    Activation("tanh"),
    Dense(32),
    Activation("tanh"),
    Dense(64),
    Activation("tanh"),
    Dense(64),
    Activation("tanh"),
    Dense(vector_length),
])

In [ ]:
autoencoder.compile(optimizer="adam", 
                    loss="mean_squared_error")

In [ ]:
autoencoder.fit(X_train, X_train,
                epochs=5000,
                batch_size=128,
                shuffle=True,
                validation_data=(X_validate, X_validate))

In [ ]:
from keras.losses import mean_squared_error, logcosh
from keras import backend as K

mse_train = K.eval(mean_squared_error(X_train, 
                                      autoencoder.predict(X_train)))

In [ ]:
mse_validate = K.eval(mean_squared_error(X_validate,
                                         autoencoder.predict(X_validate)))

In [ ]:
mse_test = K.eval(mean_squared_error(X_test, 
                                     autoencoder.predict(X_test)))

In [ ]:
mse_full = K.eval(mean_squared_error(full_dataset_df, 
                                     autoencoder.predict(full_dataset_df)))

In [ ]:
plt.boxplot([mse_train, mse_validate, mse_test, mse_full], 
            labels=["train", "validation", "test", "full"])
plt.show()

In [ ]:
import random

random_index = random.randint(0, X_test.shape[0])
plt.scatter(range(768), X_test[random_index:random_index + 1], c="b")
plt.scatter(range(768), autoencoder.predict(X_test[random_index:random_index + 1]), c="r")

In [ ]:
np.mean(mse_full), np.min(mse_full), np.max(mse_full)

In [ ]:
np.mean(mse_train), np.min(mse_train), np.max(mse_train)

In [ ]:
np.mean(mse_test), np.min(mse_test), np.max(mse_test)

In [ ]:
from datetime import datetime

current_datetime = datetime.now().strftime("%Y%m%d%H%M%S")
autoencoder.save(
    "./model/autoencoder-blue-veil-%s.h5" % current_datetime)

## Checking the outliers

In [ ]:
full_df = full_dataset_df.copy()
full_df["mse"] = mse_full
full_df.head()

In [ ]:
from skimage.io.collection import ImageCollection

def display_filtered(collection):
    for i in range(0, n_elements, 6):
        subcollection = collection[i:i + 6]
        fig, axs = plt.subplots(2, 3, figsize=(16., 10.))
        axs = axs.flatten()
        for img, ax in zip(subcollection, axs):
            ax.axis("off")
            ax.imshow(img)
        fig.show()

def display_n_smallest(df, n_elements=100):
    collection = ImageCollection(df.nsmallest(n_elements, 
                                              "mse").index)
    display_filtered(collection)

def display_n_largest(df, n_elements=100):
    collection = ImageCollection(df.nlargest(n_elements, 
                                             "mse").index)
    display_filtered(collection)

In [ ]:
display_n_smallest(full_df)

In [ ]:
test_df = pd.DataFrame(X_test)
test_df["mse"] = mse_test
test_df.head()

In [ ]:
display_n_smallest(test_df)

In [ ]:
display_n_largest(test_df, 18)

## Threshold

In [ ]:
isic_training_df = pd.read_csv(
    "./data/ISIC_2019_Training_GroundTruth.csv")
isic_training_df[1.0 != isic_training_df["MEL"]]

In [ ]:
limited_df = isic_training_df[1.0 != isic_training_df["MEL"]]

full_train_filenames, full_train_dataset = [], []
for image_filename in limited_df["image"]:
    try:
        img_path = "./data/ISIC_2019_Training_Input/%s.jpg" % image_filename
        image_rgb = io.imread(img_path)
        hist = vectorized_histogram(image_rgb)
        full_train_filenames.append(img_path)
        full_train_dataset.append(hist)
    except ValueError as e:
        print("Error while processing", img_path)
        
full_train_dataset_df = pd.DataFrame(full_train_dataset, 
                                     index=full_train_filenames)
full_train_dataset_df.to_csv("./data/ISIC_2019_Training_Input.csv")
full_train_dataset_df.head()

In [ ]:
mse_train_full = K.eval(mean_squared_error(
    full_train_dataset_df, autoencoder.predict(full_train_dataset_df)))

In [ ]:
plt.boxplot(
    [mse_train, mse_validate, mse_test, mse_full, mse_train_full], 
    labels=["train", "validation", "test", "full", "train_full"])
plt.show()

In [ ]:
np.mean(mse_train_full), np.min(mse_train_full), np.max(mse_train_full)

### Choosing the right threshold

In [ ]:
from sklearn import metrics

In [4]:
y_true = np.concatenate([
    [1] * len(X_train),
    [1] * len(X_validate),
    [1] * len(X_test),
    [0] * full_train_dataset_df.shape[0],
])
mse_all = np.concatenate([
    mse_train,
    mse_validate,
    mse_test,
    mse_train_full,
])

# curr_threshold = np.mean(mse_full)

ths = np.linspace(np.min(mse_full), 
                  np.mean(mse_full) / 2, 
                  1000)
f1_scores, precision_scores, recall_scores, accuracy_scores = [], [], [], []
for curr_threshold in ths:
    y_pred = (mse_all < curr_threshold).astype(int)
    f1_scores.append(
        metrics.f1_score(y_true, y_pred))
    precision_scores.append(
        metrics.precision_score(y_true, y_pred))
    recall_scores.append(
        metrics.recall_score(y_true, y_pred))
    accuracy_scores.append(
        metrics.accuracy_score(y_true, y_pred))

metric_scores_df = pd.DataFrame({
    "f1": f1_scores,
    "precision": precision_scores,
    "recall": recall_scores,
    "accuracy": accuracy_scores,
}, index=ths)
metric_scores_df.plot()

NameError: name 'X_train' is not defined